## Get Packages

In [235]:
using HTTP
using DataFrames
using JSON
using DataStructures
using JSONStat

## Get data from API

In [56]:
params = Dict("query" => [
        Dict(
            "code" => "Tid",
            "selection" => Dict(
                "filter" => "item",
                "values" => ["2022U34", "2022U35"]
            )
        )], "response" => Dict("format" => "json-stat2"))

Dict{String, Any} with 2 entries:
  "response" => Dict("format"=>"json-stat2")
  "query"    => Dict{String, Any}[Dict("selection"=>Dict{String, Any}("values"=…

In [57]:
url = "https://data.ssb.no/api/v0/no/table/03024/"

"https://data.ssb.no/api/v0/no/table/03024/"

In [98]:
r = HTTP.request("POST", url,
                 ["Content-Type" => "application/json"],
                 JSON.json(params))

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Cache-Control: private
Content-Type: application/json; charset=utf-8
Access-Control-Allow-Origin: *
Date: Mon, 12 Sep 2022 09:43:26 GMT
Content-Length: 1010
X-Varnish: 1004134926
Age: 0
Via: 1.1 varnish (Varnish/6.0)
Content-Security-Policy: upgrade-insecure-requests
Expect-CT: enforce, max-age=3600
Referrer-Policy: no-referrer-when-downgrade
Strict-Transport-Security: max-age=86400
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
X-XSS-Protection: 1; mode=block
X-Cache: MISS
X-UA-Compatible: IE=Edge
Accept-Ranges: bytes
Connection: keep-alive
Set-Cookie: ssbno=ffffffffc2a8b2e245525d5f4f58455e445a4a423660;expires=Mon, 12-Sep-2022 10:43:27 GMT;path=/;secure;httponly

{"class":"dataset","label":"03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke","source":"Statistisk sentralbyrå","updated":"2022-09-07T06:00:00Z","id":["VareGrupper2","ContentsCode","Tid"],"size":[2,2,2],"dimension":{"VareGrupper2":{"label":"va

In [101]:
bodyjs = JSON.parse(String(r.body))

Dict{String, Any} with 11 entries:
  "label"     => "03024: Eksport av oppalen laks, etter varegruppe, statistikkv…
  "role"      => Dict{String, Any}("time"=>Any["Tid"], "metric"=>Any["ContentsC…
  "id"        => Any["VareGrupper2", "ContentsCode", "Tid"]
  "value"     => Any[24160, 24010, 64.62, 61.94, 719, 585, 86.83, 87.02]
  "version"   => "2.0"
  "updated"   => "2022-09-07T06:00:00Z"
  "class"     => "dataset"
  "dimension" => Dict{String, Any}("ContentsCode"=>Dict{String, Any}("label"=>"…
  "size"      => Any[2, 2, 2]
  "extension" => Dict{String, Any}("px"=>Dict{String, Any}("infofile"=>"None", …
  "source"    => "Statistisk sentralbyrå"

## More to come...

In [240]:
testing = readjsondataset(JSON.parse(String(r.body)))

LoadError: Unexpected end of input
Line: 0
Around: ......
           ^


In [230]:
firstdict = Dict()
ident = bodyjs["id"]
data = bodyjs["value"]
dim = bodyjs["dimension"]
dsize = bodyjs["size"]

3-element Vector{Any}:
 2
 2
 2

In [233]:
for i in ident
    label = dim["$i"]["label"]
    #cat = dim["$i"]["category"]
    #firstdict["$label"] = 
    println(label)
end

varegruppe
statistikkvariabel
uke


In [234]:
bodyjs["dimension"]["VareGrupper2"]["category"]

Dict{String, Any} with 2 entries:
  "label" => Dict{String, Any}("02"=>"Frosen oppalen laks", "01"=>"Fersk oppale…
  "index" => Dict{String, Any}("02"=>1, "01"=>0)

In [187]:
function split(x, n)
         result = Vector{Vector{eltype(x)}}()
         start = firstindex(x)
         for len in n
           push!(result, x[start:(start + len - 1)])
           start += len
         end
         result
       end

split (generic function with 2 methods)

In [197]:
chunk(arr, n) = [arr[i:min(i + n - 1, end)] for i in 1:n:length(arr)]

chunk (generic function with 1 method)

In [204]:
length(bodyjs["value"])/2

4.0

In [218]:
bodyjs["value"][1:4]

4-element Vector{Any}:
 24160
 24010
    64.62
    61.94

In [188]:
test = [2,2,2,2]

4-element Vector{Int64}:
 2
 2
 2
 2

In [202]:
bodyjs["value"]

8-element Vector{Any}:
 24160
 24010
    64.62
    61.94
   719
   585
    86.83
    87.02

In [184]:
bodyjs["value"]

8-element Vector{Any}:
 24160
 24010
    64.62
    61.94
   719
   585
    86.83
    87.02

In [167]:
prod(bodyjs["size"]) / (sum(bodyjs["size"]) / length(bodyjs["size"]))

4.0

In [115]:
sum(bodyjs["size"]) / length(bodyjs["size"])

2.0

In [190]:
length(bodyjs["value"])

8

In [191]:
prod(bodyjs["size"])

8

In [193]:
length(bodyjs["value"]) == prod(bodyjs["size"])

true

In [194]:
length(bodyjs["size"])

3

In [196]:
sum(bodyjs["size"])

6

In [140]:
bodyjs["dimension"]["VareGrupper2"]["category"]

Dict{String, Any} with 2 entries:
  "label" => Dict{String, Any}("02"=>"Frosen oppalen laks", "01"=>"Fersk oppale…
  "index" => Dict{String, Any}("02"=>1, "01"=>0)

In [73]:
bodyjs["dimension"]["Tid"]["category"]["index"]

Dict{String, Any} with 2 entries:
  "2022U34" => 0
  "2022U35" => 1

In [86]:
bodyjs["dimension"]["ContentsCode"]["category"]["index"]

Dict{String, Any} with 2 entries:
  "Vekt"     => 0
  "Kilopris" => 1

In [90]:
bodyjs["dimension"]["VareGrupper2"]["category"]["label"]

Dict{String, Any} with 2 entries:
  "02" => "Frosen oppalen laks"
  "01" => "Fersk oppalen laks"

In [91]:
bodyjs["dimension"]["VareGrupper2"]["category"]["index"]

Dict{String, Any} with 2 entries:
  "02" => 1
  "01" => 0

In [96]:
bodyjs["id"]

3-element Vector{Any}:
 "VareGrupper2"
 "ContentsCode"
 "Tid"